**Setup**

Start by running the cell below to import `pandas` and import the exercise datasets.

In [1]:
import pandas as pd
csat = pd.read_csv('csat.csv')
csat.head()

,review_id,current_plan_start,days_subscribed,date_reviewed,pay_freq,level,rating,first_level
0,38538,2022-08-29,229,2023-04-15,Yearly,Premium,2,Free
1,78458,2022-04-11,279,2023-01-15,NaN,Free,5,Standard
2,16139,2022-01-19,422,2023-03-17,NaN,Free,3,Premium
3,88142,2022-02-26,454,2023-05-26,Yearly,Standard,3,Standard
4,21901,2022-05-27,84,2022-08-19,NaN,Free,4,Premium


## Example 1: Summarizing Positive Ratings

**Custom Function Version**

In the cell below, we've included code from an earlier exercise. In that exercise, we used a custom function to compute the percentage of positive ratings for each subscription level. Remind yourself of how this code works, and run the cell to view the output.

In [2]:
def positive(column,normalize):
    posRating = column.value_counts().loc[4] + column.value_counts().loc[5]
    if normalize:
        posRating = posRating / len(column)
    return posRating


csat.groupby('level')['rating'].apply(positive,normalize=True)

level
Free        0.705016
Premium     0.597956
Standard    0.306767
Name: rating, dtype: float64

**Built-in Methods: split-apply-combine**

There are a lot of ways to do this using only built-in pandas code. One approach is to use a split-apply-combine procedure.

First, we'll _split_ off the positive ratings so we can do calculations with just those.

In [3]:
csat_pos = csat[csat['rating'] > 3]
csat_pos.head()

,review_id,current_plan_start,days_subscribed,date_reviewed,pay_freq,level,rating,first_level
1,78458,2022-04-11,279,2023-01-15,NaN,Free,5,Standard
4,21901,2022-05-27,84,2022-08-19,NaN,Free,4,Premium
5,27938,2022-02-16,15,2022-03-03,NaN,Free,4,Free
7,21569,2022-02-03,214,2022-09-05,NaN,Free,5,Free
8,58380,2022-10-29,181,2023-04-28,NaN,Free,5,Free


Next, we'll *apply* the `count` summary method. In order to calculate the percentage of positive ratings, we need a count of the positive ratings divided by a count of the total ratings. Since we want a percentage for each level, we'll need to `groupby` the `level` column.

In [4]:
# get the total number of ratings for each level
total_ratings = csat.groupby('level')['rating'].count()

# get the number of positive ratings for each level
positive_ratings = csat_pos.groupby('level')['rating'].count()

# view both
print('total ratings')
print(total_ratings)
print('-----------------')
print('positive ratings')
print(positive_ratings)

total ratings
level
Free        17923
Premium      4599
Standard     7478
Name: rating, dtype: int64
-----------------
positive ratings
level
Free        12636
Premium      2750
Standard     2294
Name: rating, dtype: int64


Lastly, we need to _combine_ the two counts. We'll divide `positive_ratings` by `total_ratings` to compute percentages.

In [5]:
positive_ratings / total_ratings

level
Free        0.705016
Premium     0.597956
Standard    0.306767
Name: rating, dtype: float64

For a dataset of this size, the runtime will be essentially the same. Which way do you think is easier and clearer? This is matter of personal preference, so it is up to you!

## Example 2: Categorizing Rows

**Custom Function Version**

In an earlier exercise, we used custom functions to classify rows based on whether or not the customer had upgraded their plan. We've included the code solution below. Remind yourself of how this code works, and run the cell to view the output.

In [6]:
def upgrader(row):
    if row['first_level'] == 'Free' and row['level'] != 'Free':
        output = 'Upgrader'
    elif row['first_level'] == 'Standard' and row['level'] == 'Premium':
        output = 'Upgrader'
    elif row['first_level'] == row['level']:
        output = 'Steady'
    else:
        output = 'Downgrader'
    return output

## YOUR SOLUTION HERE ##
csat['grade'] = csat.apply(upgrader,axis=1)

# show output
csat.head()

,review_id,current_plan_start,days_subscribed,date_reviewed,pay_freq,level,rating,first_level,grade
0,38538,2022-08-29,229,2023-04-15,Yearly,Premium,2,Free,Upgrader
1,78458,2022-04-11,279,2023-01-15,NaN,Free,5,Standard,Downgrader
2,16139,2022-01-19,422,2023-03-17,NaN,Free,3,Premium,Downgrader
3,88142,2022-02-26,454,2023-05-26,Yearly,Standard,3,Standard,Steady
4,21901,2022-05-27,84,2022-08-19,NaN,Free,4,Premium,Downgrader


**Built-in Methods**

There are quite a few ways to achieve this using built-in methods. Let's walk through one possibility that only requires `.loc`!

First, let's build a Boolean mask that is `True` whenever a customer has upgraded.

In [7]:
# True in all rows where a customer started at Free and is now at a Paid level
free_to_paid = (csat['first_level'] == 'Free') & (csat['level'] != 'Free')

# True in all rows where a customer started at Standard and is now Premium
standard_to_premium = (csat['first_level'] == 'Standard') & (csat['level'] == 'Premium')

# upgraders are either free_to_paid or standard_to_premium
upgraded = free_to_paid | standard_to_premium

Next, let's start creating a new colum `grade2` that will store the results of our built-in method approach (the column `grade` was created above using our custom function).

The code below uses `.loc` to access all `upgraded` rows in this new column, and assigns the `Upgrader` value to them.

In [8]:
# the first input to .loc selects rows where upgraded is True
# the second input tells pandas we are updating/creating a 'grade2' column
csat.loc[upgraded,'grade2'] = 'Upgrader'

# preview
csat.head()

,review_id,current_plan_start,days_subscribed,date_reviewed,pay_freq,level,rating,first_level,grade,grade2
0,38538,2022-08-29,229,2023-04-15,Yearly,Premium,2,Free,Upgrader,Upgrader
1,78458,2022-04-11,279,2023-01-15,NaN,Free,5,Standard,Downgrader,NaN
2,16139,2022-01-19,422,2023-03-17,NaN,Free,3,Premium,Downgrader,NaN
3,88142,2022-02-26,454,2023-05-26,Yearly,Standard,3,Standard,Steady,NaN
4,21901,2022-05-27,84,2022-08-19,NaN,Free,4,Premium,Downgrader,NaN


Now, let's build a Boolean mask that corresponds to customers who are at the same level as their first plan. We'll also use `.loc` to update the `grade2` column

In [9]:
# True in all rows where the current level is the same as the first
steady = csat['first_level'] == csat['level']

csat.loc[steady,'grade2'] = 'Steady'

csat.head()

,review_id,current_plan_start,days_subscribed,date_reviewed,pay_freq,level,rating,first_level,grade,grade2
0,38538,2022-08-29,229,2023-04-15,Yearly,Premium,2,Free,Upgrader,Upgrader
1,78458,2022-04-11,279,2023-01-15,NaN,Free,5,Standard,Downgrader,NaN
2,16139,2022-01-19,422,2023-03-17,NaN,Free,3,Premium,Downgrader,NaN
3,88142,2022-02-26,454,2023-05-26,Yearly,Standard,3,Standard,Steady,Steady
4,21901,2022-05-27,84,2022-08-19,NaN,Free,4,Premium,Downgrader,NaN


All the remaining rows of `grade2` are currently `NaN` and should be `Downgrader`. We can use the built-in pandas method `.isna()` to locate all the NaNs, or `.fillna()` to replace them.

In [10]:
# .fillna() is a built-in method that automatically updates any NaN values to the value in the parentheses
csat['grade2'] = csat['grade2'].fillna('Downgrader')
csat.head()

,review_id,current_plan_start,days_subscribed,date_reviewed,pay_freq,level,rating,first_level,grade,grade2
0,38538,2022-08-29,229,2023-04-15,Yearly,Premium,2,Free,Upgrader,Upgrader
1,78458,2022-04-11,279,2023-01-15,NaN,Free,5,Standard,Downgrader,Downgrader
2,16139,2022-01-19,422,2023-03-17,NaN,Free,3,Premium,Downgrader,Downgrader
3,88142,2022-02-26,454,2023-05-26,Yearly,Standard,3,Standard,Steady,Steady
4,21901,2022-05-27,84,2022-08-19,NaN,Free,4,Premium,Downgrader,Downgrader
